In [4]:
import os
import firebase_admin
from firebase_admin import credentials, initialize_app, storage

import pandas as pd

# Define a variável de ambiente para o caminho do arquivo de credenciais
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'firebase-key.json'

# Inicializa o Firebase Admin, se ainda não estiver inicializado
if not firebase_admin._apps:
    cred = credentials.Certificate(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
    firebase_app = initialize_app(cred, {'storageBucket': 'line-hml-8a44d.appspot.com'})

# Inicializa o cliente do Firestore
db = firebase_admin.firestore.client()

# Inicializa o cliente do Storage e obtém uma referência ao bucket
bucket = storage.bucket()

# Agora você pode usar `bucket` para acessar o Firebase Storage bucket


In [5]:
pd.set_option('display.max_columns', None)


In [9]:
def listar_arquivos_no_bucket():
    """Lista todos os arquivos em um bucket do Firebase Storage."""
    
    # Supondo que `bucket` seja a referência ao seu bucket do Firebase Storage,
    # obtida durante a inicialização do Firebase Admin SDK
    blobs = bucket.list_blobs()

    print("Arquivos no bucket:")
    for blob in blobs:
        print(blob.name)

# Chame a função para listar os arquivos
listar_arquivos_no_bucket()


Arquivos no bucket:
0noyx.jpeg
2yro2.jpeg
4p9pd.jpeg
5tj69.jpeg
6wvrx.jpeg
8zg7f.jpeg
98hm9.jpeg
9re1j.jpeg
a7rc1.jpeg
c3ibt.jpeg
cjqok.jpeg
cqp0b.jpeg
cy7qb.jpeg
f2whb.jpeg
fedu4.jpeg
goyg3.jpeg
hm20o.jpeg
j4cz5.jpeg
kprta.jpeg
kxqnj.jpeg
mhn49.jpeg
pdcd0.jpeg
pqsaw.jpeg
s35b9.jpeg
szswo.jpeg
tpmcx.jpeg
xscn1.jpeg
zhzig.jpeg


In [8]:
def listar_usuarios_com_nome_e_email():
    usuarios_ref = db.collection('usuarios')  # Substitua 'usuarios' pelo nome real da sua coleção
    documentos = usuarios_ref.stream()

    for doc in documentos:
        dados = doc.to_dict()
        nome_completo = dados.get('nome_completo', 'Nome não informado')  # Substitua 'Nome_completo' pelo campo correto
        email = dados.get('email', 'Email não informado')  # Substitua 'email' pelo campo correto

        # Formata e exibe o nome e o email
        print(f'"{nome_completo}"\n{email}\n')

# Chama a função
listar_usuarios_com_nome_e_email()

"Teste5"
12ewdsa@sdfsd.com

"Nome não informado"
Email não informado

"Nome não informado"
Email não informado

"Aaasssa"
aaaaabbbbb@jose.com

"AAbabelado"
aabbabelado@jose.com

"Keu"
adora@jose.com

"Teste novo cadastro "
ajah@jdjd.com

"Ana"
ana@teste.com

"Teste"
anaa@teste.com

"Aron"
aron@bado.com

"B"
b@gmail.com

"Ha"
bete@teste.com

"Marcela"
borracheiro@teste.com

"Fly flyerson"
bradbach76@icloud.com

"Carol"
carolnome@teste.com

"Ana"
carolykg@gmail.com

"Tereza"
carolykg@hotmail.com

"Fernando"
caroool@carol.com

"Teste"
caroooooo@b.com

"Cleo Kimberlin"
cleo@jose.com

"Felipe"
cliente@line.com

"CREITO"
creito@teste.com

"line"
estab@teste.com

"Estabelecimento 4"
estabelecimento4@teste.com

"Gabriel"
fernanda@teste.com

"Marcela"
g@gmail.com

"Gabriel "
gabriel@teste.com

"Bairro"
h@teste.com

"B"
hs@teste.com

"N"
j@gmail.com

"José Cleiton "
jccs@jose.com

"José-Teste "
jctest@jose.com

"José C C santos"
josecc@jose.com

"José Cleiton"
josecleiton@jose.com

"June"
june@j

In [5]:
def listar_chaves_com_jose():
    colecoes = db.collections()  # Lista todas as coleções
    for colecao in colecoes:
        print(f'Nome da Coleção: {colecao.id}')
        documentos = colecao.stream()
        for documento in documentos:
            dados = documento.to_dict()
            for chave, valor in dados.items():
                if isinstance(valor, str) and '@jose' in valor:
                    print(f' - Documento ID: {documento.id}, Chave: {chave}, Valor: {valor}')

listar_chaves_com_jose()

Nome da Coleção: CNPJ
 - Documento ID: 37668182000183_false, Chave: Email, Valor: teste@jose.com
 - Documento ID: 39228312000192_false, Chave: Email, Valor: linkcorp@jose.com
 - Documento ID: 74958598000162_false, Chave: Email, Valor: testwork@jose.jose
 - Documento ID: 99025632000177_false, Chave: Email, Valor: josecleiton@jose.com
Nome da Coleção: CPF
 - Documento ID: 00155369040_false, Chave: Email, Valor: josecleiton@jose.com
 - Documento ID: 03293899048_false, Chave: Email, Valor: testwork@jose.jose
 - Documento ID: 09943190027_false, Chave: Email, Valor: teste@jose.com
 - Documento ID: 10697022005_true, Chave: Email, Valor: link77@jose.com
 - Documento ID: 17283478310_true, Chave: Email, Valor: adora@jose.com
 - Documento ID: 22041505095_true, Chave: Email, Valor: simone@jose.com
 - Documento ID: 44557944094_true, Chave: Email, Valor: aabbabelado@jose.com
 - Documento ID: 48928572037_true, Chave: Email, Valor: june@jose.com
 - Documento ID: 53224413018_true, Chave: Email, Valor: 

In [10]:
def colecao_para_dataframe(nome_colecao):
    colecao_ref = db.collection(nome_colecao)
    documentos = colecao_ref.stream()

    # Criar uma lista para armazenar os dados dos documentos
    documentos_lista = []

    # Iterar sobre cada documento e adicionar ao dicionário, em seguida, adicionar à lista
    for doc in documentos:
        doc_dict = doc.to_dict()
        doc_dict['document_id'] = doc.id  # Incluir o ID do documento, se necessário
        documentos_lista.append(doc_dict)

    # Converter a lista de dicionários em um DataFrame
    if documentos_lista:
        dataframe = pd.DataFrame(documentos_lista)
    else:
        dataframe = pd.DataFrame()  # Retorna um DataFrame vazio se não houver documentos

    return dataframe


dataframe = colecao_para_dataframe('usuarios')
dataframe


,bairro,cep,uid,data_nascimento,longitude,endereco,pushToken,complemento,apiPaymentCode,nome_completo,online,liner,betaTest,isDeviceLoggedIn,estado,latitude,telefone,cidade,numero,email,cpf,document_id,CPF_c,Email,deslogar_hash,deslogar,outraCategoria,razao_social,status,categorias,cnpj
0,Jardim Fraternidade,05870-030,sOEX7DhnwAhUCpMwsbNMhhgJ1A63,01/01/2000,-46.770504,Rua Andréa Solario,,,NaN,Teste5,False,True,True,False,SP,-23.679828,(11) 9999-9999,São Paulo,111,12ewdsa@sdfsd.com,841.903.430-48,12ewdsa@sdfsd.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kz8ybFqMIhwOthAOxPFJ,00735256454,caleo@emailcom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TPxEQo6pp4A2IejePsdv,00735256454,caleo@emailcom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vale dos Lagos,41256-300,2Q1qhRgthLT4ZQUPvmDLHDUF55t2,20/03/1999,-38.420261,Alameda Mata Atlântica II,None,,NaN,Aaasssa,False,True,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,aaaaabbbbb@jose.com,576.902.630-89,aaaaabbbbb@jose.com,NaN,NaN,0951b3b36f9e6041dbbc912aae7f55af,False,NaN,NaN,NaN,NaN,NaN
4,Vale dos Lagos,41256-300,RodBeP0uKdd17Q8oegcaHiZeMnB2,20/03/1998,-38.420261,Alameda Mata Atlântica II,None,,NaN,AAbabelado,False,True,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,aabbabelado@jose.com,445.579.440-94,aabbabelado@jose.com,NaN,NaN,050b9e806ad71c9714116d11c2d417fc,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Vila Gomes Cardim,03320-000,92jYlKup3ndihKxHQbAs42QEpa43,20/05/1888,-46.573645,Rua Emílio Mallet,None,,NaN,Marcela,False,True,True,False,SP,-23.548063,(85) 66666-6666,São Paulo,5,testeline@line.com,054.712.240-35,testeline@line.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Vila Gomes Cardim,03320-000,u0vm5w47sBZVJUYfSxvrbSpoSka2,NaN,-46.573002,Rua Emílio Mallet,ExponentPushToken[QGE66UIUu6OS_SQnpXnNic],,NaN,Carol,False,False,True,False,SP,-23.547844,(15) 45548-4848,São Paulo,77,tester@teste.com,179.200.850-39,tester@teste.com,NaN,NaN,NaN,NaN,,Salão do bairro,Ativo,[Salão de cabeleireiro/ Barbearia],03.974.743/0001-50
84,Vale dos Lagos,41256-300,2U9OuGm8QjWonIORa2SXCAJrrrG3,NaN,-38.420261,Alameda Mata Atlântica II,None,,75982968.0,Cleiton,False,False,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,testwork@jose.jose,032.938.990-48,testwork@jose.jose,NaN,NaN,e3af150ecc1f993e35860b6cfedb48dc,False,,Teste Work informática,Ativo,"[Saúde e bem estar, Bolos e doces]",74.958.598/0001-62
85,Vila Gomes Cardim,03320-000,Va9opjW3BjQcbZ69glGAhQ5cYwz1,NaN,-46.573695,Rua Emílio Mallet,None,,NaN,N,False,False,True,False,SP,-23.548181,(81) 54548-4848,São Paulo,8,yyy@teste.com,404.274.790-61,yyy@teste.com,NaN,NaN,28813873230f39e131694bcbabcdaf09,False,,N,Ativo,[Odontológica],


In [ ]:
# def criar_documento(nome_colecao, dados, doc_id=None):
#     """
#     Cria um novo documento em uma coleção especificada no Firestore.

#     :param nome_colecao: String que especifica o nome da coleção.
#     :param dados: Dicionário contendo os dados a serem inseridos no novo documento.
#     :param doc_id: String opcional que especifica o ID do documento. Se não for fornecido,
#                    o Firestore gera um ID único automaticamente.
#     """
#     colecao_ref = db.collection(nome_colecao)

#     if doc_id:
#         doc_ref = colecao_ref.document(doc_id)
#         doc_ref.set(dados)
#         print(f'Documento criado com ID: {doc_id}')
#     else:
#         doc_ref = colecao_ref.add(dados)
#         print(f'Documento criado com ID gerado pelo Firestore: {doc_ref[1].id}')
        
#     return str(doc_ref[1].id)

# def ler_todos_documentos(nome_colecao):
#     """
#     Lê e imprime todos os documentos de uma coleção especificada no Firestore.

#     :param nome_colecao: String que especifica o nome da coleção a ser lida.
#     """
#     colecao_ref = db.collection(nome_colecao)
#     documentos = colecao_ref.stream()
    
#     for doc in documentos:
#         print(f'{doc.id} => {doc.to_dict()}')

# def ler_documento(nome_colecao, doc_id):
#     """
#     Lê e imprime um documento específico de uma coleção pelo seu ID.

#     :param nome_colecao: String que especifica o nome da coleção.
#     :param doc_id: String que especifica o ID do documento a ser lido.
#     """
#     doc_ref = db.collection(nome_colecao).document(doc_id)
#     doc = doc_ref.get()
#     if doc.exists:
#         print(f'Documento encontrado: {doc.id} => {doc.to_dict()}')
#     else:
#         print('Documento não encontrado')

# def atualizar_documento(nome_colecao, doc_id, novos_dados):
#     """
#     Atualiza um documento específico em uma coleção com novos dados.

#     :param nome_colecao: String que especifica o nome da coleção.
#     :param doc_id: String que especifica o ID do documento a ser atualizado.
#     :param novos_dados: Dicionário contendo os novos dados para atualizar o documento.
#     """
#     doc_ref = db.collection(nome_colecao).document(doc_id)
#     doc_ref.update(novos_dados)
#     print(f'Documento {doc_id} atualizado com sucesso.')

# def deletar_documento(nome_colecao, doc_id):
#     """
#     Deleta um documento específico de uma coleção pelo seu ID.

#     :param nome_colecao: String que especifica o nome da coleção.
#     :param doc_id: String que especifica o ID do documento a ser deletado.
#     """
#     db.collection(nome_colecao).document(doc_id).delete()
#     print(f'Documento {doc_id} deletado com sucesso.')



___

### Exemplo

In [ ]:
# id_criado = criar_documento('CPF', {'CPF__c': '00735256454', 'Email': 'caleo@emailcom'})
# # Exemplo de uso
# nome_colecao = 'CPF'
# df = colecao_para_dataframe(nome_colecao)
# display(df[df['document_id'] == id_criado])

In [ ]:
# id_list = df[df['Email'] == 'caleo@emailcom']['document_id'].values

In [ ]:
# for id_doc in id_list:
#   deletar_documento('CPF', id_doc)
  
# nome_colecao = 'CPF'
# df = colecao_para_dataframe(nome_colecao)
# display(df)